In [1]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm

In [2]:
# Root directory for the dataset
root = "data/DressCode/"

# Map labels to their corresponding directories
DIRECTORY_MAP = ["upper_body", "lower_body", "dresses"]

# Map labels to their corresponding segmentations
SEGMENT_MAP = [[4], [5, 6], [7]]

In [3]:
# Read in the dataset
train_pairs = pd.read_csv(
    os.path.join(root, "train_pairs.txt"),
    delimiter="\t",
    header=None,
    names=["model", "garment", "label"],
)

test_pairs = pd.read_csv(
    os.path.join(root, "test_pairs_paired.txt"),
    delimiter="\t",
    header=None,
    names=["model", "garment", "label"],
)

pairs = pd.concat([train_pairs, test_pairs])

pairs.head()

,model,garment,label
0,000000_0.jpg,000000_1.jpg,0
1,000001_0.jpg,000001_1.jpg,0
2,000002_0.jpg,000002_1.jpg,0
3,000003_0.jpg,000003_1.jpg,0
4,000004_0.jpg,000004_1.jpg,0


In [4]:
# Create output directories if they don't exist
for directory in DIRECTORY_MAP:
    os.makedirs(os.path.join(root, directory, "cropped_images"), exist_ok=True)

In [5]:
def get_bounding_box(mask: np.ndarray) -> tuple[int, int, int, int]:
    """
    Get the bounding box around the mask.

    Returns (x_min, y_min, x_max, y_max): The bounding box.
    """

    x_indices, y_indices = np.where(mask)

    if len(x_indices) == 0 or len(y_indices) == 0:
        return 0, 0, mask.shape[0], mask.shape[1]

    x_min = int(np.min(x_indices))
    x_max = int(np.max(x_indices))
    y_min = int(np.min(y_indices))
    y_max = int(np.max(y_indices))

    return x_min, y_min, x_max, y_max


# Crop all the model images
for model, garment, label in tqdm(
    pairs.values, desc="Cropping Models", total=len(pairs), unit="image"
):
    # Load the model image
    model_image = Image.open(
        os.path.join(root, DIRECTORY_MAP[label], "images", model)
    ).convert("RGB")

    # Load the segmentation
    segmentation = np.array(
        Image.open(
            os.path.join(
                root, DIRECTORY_MAP[label], "label_maps", model.split("_")[0] + "_4.png"
            )
        )
    )

    # Get the mask for the label
    mask = np.isin(segmentation, SEGMENT_MAP[label])

    if not mask.any():
        print(f"Skipping model {model} as mask is empty.")
        continue

    # Get the bounding box for the mask
    x_min, y_min, x_max, y_max = get_bounding_box(mask)

    # Crop the image
    model_image_cropped = model_image.crop((y_min, x_min, y_max, x_max))

    # Save the cropped image
    model_image_cropped.save(
        os.path.join(root, DIRECTORY_MAP[label], "cropped_images", model)
    )

Cropping Models:   0%|          | 0/53792 [00:00<?, ?image/s]

Cropping Models:  20%|█▉        | 10569/53792 [01:34<06:19, 113.87image/s]

Skipping model 010560_0.jpg as mask is empty.


Cropping Models:  34%|███▎      | 18117/53792 [03:37<09:43, 61.12image/s] 

Skipping model 018112_0.jpg as mask is empty.


Cropping Models: 100%|██████████| 53792/53792 [13:40<00:00, 65.59image/s]
